# Land cover change inferred from field and remote sensing data

Based on my scientific article:

Shevtsova, I., Heim, B., Kruse, S., Schröder, J., Troeva, E., Pestryakova, L., Zakharov, E. and Herzschuh, U.: Strong shrub expansion in tundra-taiga, tree infilling in taiga and stable tundra in central Chukotka (north-eastern Siberia) between 2000 and 2017, Environmental Research Letters, 15(8), 085006, <a href="doi:10.1088/1748-9326/ab9059">doi:10.1088/1748-9326/ab9059</a>, 2020. (eng.)

### Preparation

import required libraries

In [1]:
#ecological analysis
library(permute)
library(lattice)
library(vegan)
#data visualisation
library(ggplot2)
#read data from PANGAEA repository
library(pangaear)
#scrapping data from web (my paper)
require(rvest)
require(magrittr)

This is vegan 2.6-4

Registered S3 method overwritten by 'httr':
  method           from  
  print.cache_info hoardr

Lade nötiges Paket: rvest

Lade nötiges Paket: magrittr



### Loading data

<ul>
    <li>Remote sensing data</li>
    <li>Field vegetation</li>
</ul>

**Load remote sensing (LANDSAT) data**
<ul>
    <li>Load raster data</li>
    <li>Mask water, shadows, shadow clouds, artifacts</li>
    <li>Transform to Landsat-7-like</li>
    <li>Apply topographical correction</li>
    <li>Calculate the Indices</li>
    <li>Prepare a dataset: Indices on expedition sites</li>
</ul>

Most of the actions were done in preparation steps and not covered here. I may add them here later.

**Indices on sites**

we are loading them from my paper

In [2]:
url <- "https://iopscience.iop.org/article/10.1088/1748-9326/ab9059"
paper <- read_html(url, as.data.frame=T, stringsAsFactors = TRUE)
paper %>%  
        html_nodes("table") %>% 
        .[[3]] %>% 
        html_table(fill=T) -> landsat_indices
landsat_indices

Index Site,NDVI,NDWI,NDSI,Index Site,NDVI,NDWI,NDSI
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
V01,0.72,−0.64,0.88,V42,0.43,−0.48,0.69
V02,0.64,−0.60,0.91,V43,0.37,−0.41,0.73
V03,0.70,−0.63,0.90,V44,0.34,−0.40,0.58
V04,0.77,−0.67,0.81,V45,0.59,−0.56,0.74
V05,0.74,−0.66,0.88,V46,0.63,−0.61,0.73
V06,0.67,−0.61,1.00,V47,0.65,−0.63,0.69
V07,0.33,−0.36,0.87,V48,0.63,−0.63,0.74
V08,0.45,−0.48,0.94,V49,0.61,−0.61,0.73
V09,0.68,−0.63,0.82,V50,0.49,−0.51,0.63


Now let us restructure these data in a usable format

In [3]:
landsat_indices_long <- rbind(landsat_indices[,1:4],landsat_indices[,5:8])
landsat_indices_long

Index Site,NDVI,NDWI,NDSI
<chr>,<dbl>,<chr>,<chr>
V01,0.72,−0.64,0.88
V02,0.64,−0.60,0.91
V03,0.70,−0.63,0.90
V04,0.77,−0.67,0.81
V05,0.74,−0.66,0.88
V06,0.67,−0.61,1.00
V07,0.33,−0.36,0.87
V08,0.45,−0.48,0.94
V09,0.68,−0.63,0.82


In [4]:
colnames(landsat_indices_long) <- c("Site", "NDVI", "NDWI", "NDSI")

before we do analysis further, we need to take a look on a map and location of the sites. Some of them might be in a shadow and we would not want them to be in the model. The coordinates of all sites are given together with the expeditions' vegetation data. Let's get it first.

**Retrieve the vegeation field data by number from PANGAEA repo and quick check**

The link to the dataset:
Shevtsova, Iuliia; Herzschuh, Ulrike; Heim, Birgit; Kruse, Stefan; Schröder, Julius; Troeva, Elena I; Pestryakova, Luidmila A; Zakharov, Evgenii S (2019): Foliage projective cover of 57 vegetation sites of central Chukotka from 2016. Alfred Wegener Institute - Research Unit Potsdam, PANGAEA, <a href="https://doi.org/10.1594/PANGAEA.908570">https://doi.org/10.1594/PANGAEA.908570</a>

In [5]:
veg_repo <- pg_data(doi = '10.1594/PANGAEA.908570')
veg_repo <- veg_repo[[1]]
veg_data <- veg_repo$data
#check
veg_data[1:3,]
colnames(veg_data)


Processing 1 files



Event,Site,Latitude,Longitude,Code (region),Plot size (in m),Plot rad [m],Elevation [m a.s.l.],Slope (slope in m/10m),Aspect,⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16-KP-01-V01,V01,67.3618,168.2542,16-KP-01,,15,492,2.50,S,⋯,0.4,0.0,0.2,0,0,0.3,0,0,0.0,0.0
16-KP-01-V02,V02,67.3660,168.2366,16-KP-01,,15,559,1.67,S,⋯,0.3,0.2,0.7,0,0,0.0,0,0,0.1,0.2
16-KP-01-V03,V03,67.3664,168.2948,16-KP-01,24x26,NA,485,1.43,SW,⋯,0.1,0.1,0.4,0,0,0.0,0,0,2.0,0.0


[1] "Event"                              "Site"                              
 [3] "Latitude"                           "Longitude"                         
 [5] "Code (region)"                      "Plot size (in m)"                  
 [7] "Plot rad [m]"                       "Elevation [m a.s.l.]"              
 [9] "Slope (slope in m/10m)"             "Aspect"                            
[11] "A. fruticosa cov [%]"               "P. pumila cov [%]"                 
[13] "L. cajanderii cov [%]"              "Salix cov [%] (non-creeping)"      
[15] "Salix cov [%] (creeping)"           "B. exilis cov [%]"                 
[17] "C. tetragona cov [%]"               "E. nigrum cov [%]"                 
[19] "L. palustre cov [%]"                "V. uliginosum cov [%]"             
[21] "V. vitis-idaea [%]"                 "R. adamsii cov [%]"                
[23] "R. arctica cov [%]"                 "A. alpina cov [%]"                 
[25] "Lichen cov [%]"                     "Moss cov [%]"                      
[27] "Poaceae+Cyperaceae cov [%]"         "Asteraceae cov [%]"                
[29] "Brassicaceae cov [%]"               "Campanulaceae cov [%]"             
[31] "Caprifoliaceae cov [%]"             "Caryophyllaceae+Montiaceae cov [%]"
[33] "Equisetaceae cov [%]"               "Fabaceae cov [%]"                  
[35] "Gentianaceae cov [%]"               "Grossulariaceae cov [%]"           
[37] "Orobanchaceae cov [%]"              "Orthilia cov [%]"                  
[39] "Polygonaceae cov [%]"               "Primulaceae cov [%]"               
[41] "Pyrola spp. cov [%] (other)"        "Ranunculaceae cov [%]"             
[43] "Rosaceae cov [%] (other)"           "D. octopetala cov [%]"             
[45] "Rubiaceae cov [%]"                  "Saxifragaceae cov [%]"

In [6]:
#cleaning last row with no data
veg_data[58,]
veg_data <- veg_data[-58,]

#check
veg_data[1:3,]
colnames(veg_data)

Event,Site,Latitude,Longitude,Code (region),Plot size (in m),Plot rad [m],Elevation [m a.s.l.],Slope (slope in m/10m),Aspect,⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16-KP-04-V58,,NA,NA,,,NA,NA,NA,,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Event,Site,Latitude,Longitude,Code (region),Plot size (in m),Plot rad [m],Elevation [m a.s.l.],Slope (slope in m/10m),Aspect,⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16-KP-01-V01,V01,67.3618,168.2542,16-KP-01,,15,492,2.50,S,⋯,0.4,0.0,0.2,0,0,0.3,0,0,0.0,0.0
16-KP-01-V02,V02,67.3660,168.2366,16-KP-01,,15,559,1.67,S,⋯,0.3,0.2,0.7,0,0,0.0,0,0,0.1,0.2
16-KP-01-V03,V03,67.3664,168.2948,16-KP-01,24x26,NA,485,1.43,SW,⋯,0.1,0.1,0.4,0,0,0.0,0,0,2.0,0.0


[1] "Event"                              "Site"                              
 [3] "Latitude"                           "Longitude"                         
 [5] "Code (region)"                      "Plot size (in m)"                  
 [7] "Plot rad [m]"                       "Elevation [m a.s.l.]"              
 [9] "Slope (slope in m/10m)"             "Aspect"                            
[11] "A. fruticosa cov [%]"               "P. pumila cov [%]"                 
[13] "L. cajanderii cov [%]"              "Salix cov [%] (non-creeping)"      
[15] "Salix cov [%] (creeping)"           "B. exilis cov [%]"                 
[17] "C. tetragona cov [%]"               "E. nigrum cov [%]"                 
[19] "L. palustre cov [%]"                "V. uliginosum cov [%]"             
[21] "V. vitis-idaea [%]"                 "R. adamsii cov [%]"                
[23] "R. arctica cov [%]"                 "A. alpina cov [%]"                 
[25] "Lichen cov [%]"                     "Moss cov [%]"                      
[27] "Poaceae+Cyperaceae cov [%]"         "Asteraceae cov [%]"                
[29] "Brassicaceae cov [%]"               "Campanulaceae cov [%]"             
[31] "Caprifoliaceae cov [%]"             "Caryophyllaceae+Montiaceae cov [%]"
[33] "Equisetaceae cov [%]"               "Fabaceae cov [%]"                  
[35] "Gentianaceae cov [%]"               "Grossulariaceae cov [%]"           
[37] "Orobanchaceae cov [%]"              "Orthilia cov [%]"                  
[39] "Polygonaceae cov [%]"               "Primulaceae cov [%]"               
[41] "Pyrola spp. cov [%] (other)"        "Ranunculaceae cov [%]"             
[43] "Rosaceae cov [%] (other)"           "D. octopetala cov [%]"             
[45] "Rubiaceae cov [%]"                  "Saxifragaceae cov [%]"

**Check for differences in datasets**

In [7]:
colnames(veg_data[2]) <- "Site"
#first let us check if we have the same amount of sites in both datasets
print("vegetation data:")
nrow(veg_data)
print("Landsat data:")
nrow(landsat_indices_long)

[1] "vegetation data:"


[1] 57

[1] "Landsat data:"


[1] 52

In [8]:
#we see that this is not a case. Let us find the sites which are not in both datasets and delete them. 
print("all vegetation Sites")
veg_data$Site
print("all Landsat Sites")
sort(landsat_indices_long$Site)

print("Only Sites present in both datasets")
is <- intersect(veg_data$Site,landsat_indices_long$Site)
is

[1] "all vegetation Sites"


[1] "V01" "V02" "V03" "V04" "V05" "V06" "V07" "V08" "V09" "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V18" "V19" "V20" "V21" "V22" "V23" "V24" "V25"
[25] "V26" "V27" "V28" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36" "V37"
[37] "V38" "V39" "V40" "V41" "V42" "V43" "V44" "V45" "V46" "V47" "V48" "V49"
[49] "V50" "V51" "V52" "V53" "V54" "V55" "V56" "V57" "V58"

[1] "all Landsat Sites"


[1] "V01" "V02" "V03" "V04" "V05" "V06" "V07" "V08" "V09" "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V18" "V19" "V20" "V21" "V22" "V23" "V25" "V26"
[25] "V27" "V28" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36" "V37" "V38"
[37] "V39" "V40" "V41" "V42" "V43" "V44" "V45" "V46" "V47" "V48" "V49" "V50"
[49] "V51" "V54" "V57" "V58"

[1] "Only Sites present in both datasets"


[1] "V01" "V02" "V03" "V04" "V05" "V06" "V07" "V08" "V09" "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V18" "V19" "V20" "V21" "V22" "V23" "V25" "V26"
[25] "V27" "V28" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36" "V37" "V38"
[37] "V39" "V40" "V41" "V42" "V43" "V44" "V45" "V46" "V47" "V48" "V49" "V50"
[49] "V51" "V54" "V57" "V58"

In [9]:
#Adjust data in both datasets
veg_data <- veg_data[veg_data$Site %in% is,]
landsat_indices_long <- landsat_indices_long[landsat_indices_long$Site %in% is,]

#Check
print("all vegetation Sites")
veg_data$Site
print("all Landsat Sites")
landsat_indices_long$Site

[1] "all vegetation Sites"


[1] "V01" "V02" "V03" "V04" "V05" "V06" "V07" "V08" "V09" "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V18" "V19" "V20" "V21" "V22" "V23" "V25" "V26"
[25] "V27" "V28" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36" "V37" "V38"
[37] "V39" "V40" "V41" "V42" "V43" "V44" "V45" "V46" "V47" "V48" "V49" "V50"
[49] "V51" "V54" "V57" "V58"

[1] "all Landsat Sites"


[1] "V01" "V02" "V03" "V04" "V05" "V06" "V07" "V08" "V09" "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V18" "V19" "V20" "V21" "V22" "V23" "V25" "V26"
[25] "V27" "V28" "V42" "V43" "V44" "V45" "V46" "V47" "V48" "V49" "V50" "V51"
[37] "V54" "V57" "V58" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36" "V37"
[49] "V38" "V39" "V40" "V41"

In [10]:
#order landsat sites
landsat_indices_long <- landsat_indices_long[order(landsat_indices_long$Site),]

In [11]:
#finally merge datasets' parts to get the coordinates on Landsat data on the site, which are usable (we will use it later on)
landsat_coord <- cbind(landsat_indices_long,veg_data[,c(3,4,8,9,10)])
landsat_coord[1:3,]

,Site,NDVI,NDWI,NDSI,Latitude,Longitude,Elevation [m a.s.l.],Slope (slope in m/10m),Aspect
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<chr>
1,V01,0.72,−0.64,0.88,67.3618,168.2542,492,2.50,S
2,V02,0.64,−0.60,0.91,67.3660,168.2366,559,1.67,S
3,V03,0.70,−0.63,0.90,67.3664,168.2948,485,1.43,SW


### Vegetation data preanalysis

In [12]:
veg_taxa <- veg_data[,c(2,11:46)]
veg_taxa[1:3,]

Site,A. fruticosa cov [%],P. pumila cov [%],L. cajanderii cov [%],Salix cov [%] (non-creeping),Salix cov [%] (creeping),B. exilis cov [%],C. tetragona cov [%],E. nigrum cov [%],L. palustre cov [%],⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
V01,0.9,0.0,10,0.9,0.2,8.3,0,3.4,4.3,⋯,0.4,0.0,0.2,0,0,0.3,0,0,0.0,0.0
V02,1.0,0.5,4,1.0,0.6,4.2,0,3.0,7.8,⋯,0.3,0.2,0.7,0,0,0.0,0,0,0.1,0.2
V03,0.0,0.5,8,0.0,0.0,6.0,0,3.0,8.8,⋯,0.1,0.1,0.4,0,0,0.0,0,0,2.0,0.0


**Hellinger transformation**

The square root of observed values that have been devided by row(site) sums.
IN R: "decostand" function (vegan).(Legendre & Gallagher, 2001)
Recommendended for clustering or ordination of species abundance.
    
Purpose: removes distances in total abundance, bur maintains differences in relative species composition among sites.

In [13]:
veg_hel_transf <- decostand(veg_taxa[,-1], method = "hellinger", MARGIN=1)
veg_hel_transf[1:3,]

,A. fruticosa cov [%],P. pumila cov [%],L. cajanderii cov [%],Salix cov [%] (non-creeping),Salix cov [%] (creeping),B. exilis cov [%],C. tetragona cov [%],E. nigrum cov [%],L. palustre cov [%],V. uliginosum cov [%],⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1280369,0.00000000,0.4267896,0.1280369,0.06035716,0.3888238,0,0.2488590,0.2798647,0.3017858,⋯,0.08535792,0.00000000,0.06035716,0,0,0.07392213,0,0,0.00000000,0.00000000
2,0.1014301,0.07172191,0.2028602,0.1014301,0.07856742,0.2078699,0,0.1756821,0.2832789,0.1504452,⋯,0.05555556,0.04536092,0.08486251,0,0,0.00000000,0,0,0.03207501,0.04536092
3,0.0000000,0.07224408,0.2889763,0.0000000,0.00000000,0.2502608,0,0.1769611,0.3030809,0.3065057,⋯,0.03230853,0.03230853,0.06461707,0,0,0.00000000,0,0,0.14448815,0.00000000


**Detrended correspondence Analysis**

Needed to know if the field data distributed on a short linear gradient or is it more a unimodal distribution.
In R: function "decorana".

In [14]:
DCA_res <- decorana(veg_hel_transf)
DCA_res


Call:
decorana(veg = veg_hel_transf) 

Detrended correspondence analysis with 26 segments.
Rescaling of axes with 4 iterations.
Total inertia (scaled Chi-square): 1.6412 

                       DCA1  DCA2   DCA3    DCA4
Eigenvalues          0.3084 0.229 0.1295 0.10349
Additive Eigenvalues 0.3084 0.229 0.1277 0.10095
Decorana values      0.3108 0.205 0.1287 0.05604
Axis lengths         2.5787 2.191 2.0058 1.94020


The length of the first detrended correspondence analysis axis is equal to **2.58 standard deviation units** meaning the field data in our case is distributed on a short linear gradient rather than a unimodal distribution. 

## Analysis of vegetation & Landsat data, their relationship, ordination in RDA space

In [15]:
#Look at the data first
rownames(veg_hel_transf) <- veg_data$Site

landsat_indices_long <- as.data.frame(landsat_indices_long)
rownames(landsat_indices_long) <- landsat_indices_long$Site

veg_hel_transf[1:3,]
landsat_indices_long[1:3,]

,A. fruticosa cov [%],P. pumila cov [%],L. cajanderii cov [%],Salix cov [%] (non-creeping),Salix cov [%] (creeping),B. exilis cov [%],C. tetragona cov [%],E. nigrum cov [%],L. palustre cov [%],V. uliginosum cov [%],⋯,Orobanchaceae cov [%],Orthilia cov [%],Polygonaceae cov [%],Primulaceae cov [%],Pyrola spp. cov [%] (other),Ranunculaceae cov [%],Rosaceae cov [%] (other),D. octopetala cov [%],Rubiaceae cov [%],Saxifragaceae cov [%]
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
V01,0.1280369,0.00000000,0.4267896,0.1280369,0.06035716,0.3888238,0,0.2488590,0.2798647,0.3017858,⋯,0.08535792,0.00000000,0.06035716,0,0,0.07392213,0,0,0.00000000,0.00000000
V02,0.1014301,0.07172191,0.2028602,0.1014301,0.07856742,0.2078699,0,0.1756821,0.2832789,0.1504452,⋯,0.05555556,0.04536092,0.08486251,0,0,0.00000000,0,0,0.03207501,0.04536092
V03,0.0000000,0.07224408,0.2889763,0.0000000,0.00000000,0.2502608,0,0.1769611,0.3030809,0.3065057,⋯,0.03230853,0.03230853,0.06461707,0,0,0.00000000,0,0,0.14448815,0.00000000


,Site,NDVI,NDWI,NDSI
,<chr>,<dbl>,<chr>,<chr>
V01,V01,0.72,−0.64,0.88
V02,V02,0.64,−0.60,0.91
V03,V03,0.70,−0.63,0.90


**Redundancy analysis (RDA)**

In [17]:
#coming soon...